In [1]:
import os
import sys
import urllib.request
import datetime
import time
import json

### 뉴스 데이터 가져오기

In [2]:
client_id = 'x9KOW5id6lADbA41jdlT'
client_secret = 'A5c7fR_aYm'

In [3]:
def getRequestUrl(url):
    req = urllib.request.Request(url)
    req.add_header("X-Naver-Client-Id", client_id)
    req.add_header("X-Naver-Client-Secret", client_secret)

    try:
        response = urllib.request.urlopen(req)
        if response.getcode() == 200:
            print("[%s] Url Request Success" % datetime.datetime.now())
            return response.read().decode('utf-8')
    except Exception as e:
        print(e)
        print("[%s] Error for URL : %s" % (datetime.datetime.now(), url))
        return None

In [4]:
def getNaverSearch(node, srcText, start, display):
    base = "https://openapi.naver.com/v1/search"
    node = "/%s.json" % node
    parameters = "?query=%s&start=%s&display=%s" % (urllib.parse.quote(srcText), start, display)

    url = base + node + parameters
    responseDecode = getRequestUrl(url) #[CODE 1]

    if (responseDecode == None):
        return None
    else:
        return json.loads(responseDecode)

In [5]:
def getPostData(post, jsonResult, cnt):
    title = post['title']
    description = post['description']
    org_link = post['originallink']
    link = post['link']

    pDate = datetime.datetime.strptime(post['pubDate'], '%a, %d %b %Y %H:%M:%S +0900')
    pDate = pDate.strftime('%Y-%m-%d %H:%M:%S')

    jsonResult.append({'cnt':cnt, 'title':title, 'description': description, 'org_link':org_link, 'link': org_link, 'pDate':pDate})
    return

In [6]:
node = 'news' #크롤링할 대상
srcText = input('검색어를 입력하세요: ')
cnt = 0
jsonResult = []
jsonResponse = getNaverSearch(node, srcText, 1, 100) #[CODE 2]
total = jsonResponse['total']

while ((jsonResponse != None) and (jsonResponse['display'] != 0)):
    for post in jsonResponse['items']:
        cnt += 1
        getPostData(post, jsonResult, cnt) #[CODE 3]
    
    start = jsonResponse['start'] + jsonResponse['display']
    jsonResponse = getNaverSearch(node, srcText, start, 100) #[CODE 2]

print('전체 검색 : %d 건' %total)

with open('%s_naver_%s.json' % (srcText, node), 'w', encoding='utf8') as outfile:
    jsonFile = json.dumps(jsonResult, indent = 4, sort_keys = True, ensure_ascii = False)
    outfile.write(jsonFile)

print("가져온 데이터 : %d 건" %(cnt))
print('%s_naver_%s.json SAVED' % (srcText, node))

검색어를 입력하세요: 현대차
[2022-05-31 13:39:24.431590] Url Request Success
[2022-05-31 13:39:25.645451] Url Request Success
[2022-05-31 13:39:26.868818] Url Request Success
[2022-05-31 13:39:28.114514] Url Request Success
[2022-05-31 13:39:29.362961] Url Request Success
[2022-05-31 13:39:30.562816] Url Request Success
[2022-05-31 13:39:31.784827] Url Request Success
[2022-05-31 13:39:33.008403] Url Request Success
[2022-05-31 13:39:34.248303] Url Request Success
[2022-05-31 13:39:35.490974] Url Request Success
HTTP Error 400: Bad Request
[2022-05-31 13:39:36.408092] Error for URL : https://openapi.naver.com/v1/search/news.json?query=%ED%98%84%EB%8C%80%EC%B0%A8&start=1001&display=100
전체 검색 : 1345703 건
가져온 데이터 : 1000 건
현대차_naver_news.json SAVED


### 가져온 데이터를 데이터 프레임화 하기

In [7]:
import pandas as pd
import re

In [8]:
def clean_html(x):
  #https://predictor-ver1.tistory.com/4
  x = re.sub("\&\w*\;","",x)
  x = re.sub("<.*?>","",x)
  return x

In [9]:
df_json = pd.DataFrame(jsonResult)
df_json.head()

cnt                                              title  \
0    1      부산엑스포 민간위 공식 출범…최태원 &quot;정부-기업은 원팀&quot;(종합)   
1    2                    <b>현대차</b>, 'H-모빌리티 클래스' 참가자 모집   
2    3   '웨이브 이틀째 초강세 보여' 비트코인 3만1700달러..미 선물 하락세 속 이더...   
3    4  코스피, 외인 매수에 상승 마감…LG화학, 네이버, 카카오 상승 VS 삼성SDI, ...   
4    5                 &quot;2030부산엑스포 유치에 모든 역량 결집&quot;   

                                         description  \
0  기사내용 요약尹, 출범식 참석해 기업인들 격려 삼성·<b>현대차</b>·SK·LG·...   
1  (사진=<b>현대차</b>) 현대자동차가 최근 미래 모빌리티 산업 인재를 양성하기 ...   
2  외국인이 SK하이닉스를 천억 넘게 순매수한 가운데 시가총액 상위 종목들을 보면 LG...   
3  03%), 삼성전자(-0.44%), 삼성전자우(-0.33%)를 제외하고 상승 마감했...   
4  대한상공회의소 최태원 회장이 위원장을 맡았고, 삼성과 SK, <b>현대차</b>, ...   

                                            org_link  \
0  http://www.newsis.com/view/?id=NISX20220531_00...   
1  https://www.asiatime.co.kr/article/20220531500369   
2      https://idsn.co.kr/news/view/1065615503672069   
3  http://www.econonews.co.kr/news/articleView.ht...   
4  https://busanmbc.co.kr/article/poppzLV7g1rsJyM...   

                                                link                pDate  
0  http://www.newsis.com/view/?id=NISX20220531_00...  2022-05-31 22:22:00  
1  https://www.asiatime.co.kr/article/20220531500369  2022-05-31 22:10:00  
2      https://idsn.co.kr/news/view/1065615503672069  2022-05-31 21:46:00  
3  http://www.econonews.co.kr/news/articleView.ht...  2022-05-31 21:32:00  
4  https://busanmbc.co.kr/article/poppzLV7g1rsJyM...  2022-05-31 21:12:00

In [10]:
df_json['title'] = df_json['title'].apply(lambda x: clean_html(x))
df_json['description'] = df_json['description'].apply(lambda x: clean_html(x))
df_json.head()

cnt                                              title  \
0    1                  부산엑스포 민간위 공식 출범…최태원 정부-기업은 원팀(종합)   
1    2                           현대차, 'H-모빌리티 클래스' 참가자 모집   
2    3   '웨이브 이틀째 초강세 보여' 비트코인 3만1700달러..미 선물 하락세 속 이더...   
3    4  코스피, 외인 매수에 상승 마감…LG화학, 네이버, 카카오 상승 VS 삼성SDI, ...   
4    5                             2030부산엑스포 유치에 모든 역량 결집   

                                         description  \
0  기사내용 요약尹, 출범식 참석해 기업인들 격려 삼성·현대차·SK·LG·롯데 등 11...   
1  (사진=현대차) 현대자동차가 최근 미래 모빌리티 산업 인재를 양성하기 위한 교육 프...   
2  외국인이 SK하이닉스를 천억 넘게 순매수한 가운데 시가총액 상위 종목들을 보면 LG...   
3  03%), 삼성전자(-0.44%), 삼성전자우(-0.33%)를 제외하고 상승 마감했...   
4  대한상공회의소 최태원 회장이 위원장을 맡았고, 삼성과 SK, 현대차, LG 등 국내...   

                                            org_link  \
0  http://www.newsis.com/view/?id=NISX20220531_00...   
1  https://www.asiatime.co.kr/article/20220531500369   
2      https://idsn.co.kr/news/view/1065615503672069   
3  http://www.econonews.co.kr/news/articleView.ht...   
4  https://busanmbc.co.kr/article/poppzLV7g1rsJyM...   

                                                link                pDate  
0  http://www.newsis.com/view/?id=NISX20220531_00...  2022-05-31 22:22:00  
1  https://www.asiatime.co.kr/article/20220531500369  2022-05-31 22:10:00  
2      https://idsn.co.kr/news/view/1065615503672069  2022-05-31 21:46:00  
3  http://www.econonews.co.kr/news/articleView.ht...  2022-05-31 21:32:00  
4  https://busanmbc.co.kr/article/poppzLV7g1rsJyM...  2022-05-31 21:12:00

In [11]:
df_json['description'].head()

0    기사내용 요약尹, 출범식 참석해 기업인들 격려 삼성·현대차·SK·LG·롯데 등 11...
1    (사진=현대차) 현대자동차가 최근 미래 모빌리티 산업 인재를 양성하기 위한 교육 프...
2    외국인이 SK하이닉스를 천억 넘게 순매수한 가운데 시가총액 상위 종목들을 보면 LG...
3    03%), 삼성전자(-0.44%), 삼성전자우(-0.33%)를 제외하고 상승 마감했...
4    대한상공회의소 최태원 회장이 위원장을 맡았고, 삼성과 SK, 현대차, LG 등 국내...
Name: description, dtype: object

In [12]:
!pip install konlpy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 19.4 MB 1.3 MB/s 
     |████████████████████████████████| 453 kB 70.6 MB/s 


In [13]:
from konlpy.tag import Kkma, Komoran, Okt

okt = Okt()
kkm = Kkma()
kom = Komoran()

In [14]:
description_list = df_json['description'].values.tolist()
description_slice = []
for i in range(0, len(description_list)):
    description_slice += description_list[i].split(' ')
len(description_slice)

25733

In [15]:
title_list = df_json['title'].values.tolist()
title_slice = []
for i in range(0, len(title_list)):
    title_slice += title_list[i].split(' ')
len(title_slice)

7102

In [16]:
from sklearn.feature_extraction.text import TfidfVectorizer

vect = TfidfVectorizer(min_df = 50, ngram_range = (1, 3))
vect.fit(description_list)
vect.vocabulary_
sorted(vect.vocabulary_.items())

[('07', 0),
 ('2027년까지', 1),
 ('24시', 2),
 ('250개', 3),
 ('30일', 4),
 ('a1', 5),
 ('cj', 6),
 ('lg', 7),
 ('lg 롯데', 8),
 ('lg에너지솔루션', 9),
 ('lg화학', 10),
 ('naver', 11),
 ('sk', 12),
 ('sk lg', 13),
 ('sk lg 롯데', 14),
 ('sk 현대차', 15),
 ('sk하이닉스', 16),
 ('고성능', 17),
 ('공영운', 18),
 ('교육', 19),
 ('국내', 20),
 ('국내 스타트업', 21),
 ('글로벌', 22),
 ('기아', 23),
 ('뉘르부르크링', 24),
 ('뉘르부르크링 24시', 25),
 ('다운타운', 26),
 ('다운타운 미래', 27),
 ('대상으로', 28),
 ('대학', 29),
 ('대학 생을', 30),
 ('등이', 31),
 ('따르면', 32),
 ('롯데', 33),
 ('맞은', 34),
 ('모빌리티', 35),
 ('모빌리티 클래스', 36),
 ('모터스튜디오', 37),
 ('미래', 38),
 ('미래 모빌리티', 39),
 ('분야의', 40),
 ('브랜드', 41),
 ('사장', 42),
 ('사진', 43),
 ('삼성', 44),
 ('삼성sdi', 45),
 ('삼성바이오로직스', 46),
 ('삼성전자', 47),
 ('삼성전자 현대차', 48),
 ('생을', 49),
 ('생태계', 50),
 ('서울', 51),
 ('스타트업', 52),
 ('스타트업 생태계', 53),
 ('시가총액', 54),
 ('엘란트라', 55),
 ('오는', 56),
 ('오는 2027년까지', 57),
 ('온드림', 58),
 ('위해', 59),
 ('육성', 60),
 ('이공계', 61),
 ('이공계 대학', 62),
 ('이공계 대학 생을', 63),
 ('일자리', 64),
 ('있다', 65),
 ('재단이',

In [17]:
vect2 = TfidfVectorizer(min_df = 20, ngram_range = (1, 2))
vect2.fit(title_list)
vect2.vocabulary_
sorted(vect2.vocabulary_.items())

[('1000억', 0),
 ('1000억 투자', 1),
 ('1000억원', 2),
 ('24시', 3),
 ('24시 내구레이스', 4),
 ('250개', 5),
 ('5년간', 6),
 ('7년', 7),
 ('7년 연속', 8),
 ('lg', 9),
 ('sk', 10),
 ('공식', 11),
 ('공식 출범', 12),
 ('구축', 13),
 ('국내', 14),
 ('국내 스타트업', 15),
 ('글로벌', 16),
 ('기관', 17),
 ('내구레이스', 18),
 ('내구레이스 7년', 19),
 ('뉘르부르크링', 20),
 ('뉘르부르크링 24시', 21),
 ('마감', 22),
 ('메타버스', 23),
 ('메타버스 플랫폼', 24),
 ('모빌리티', 25),
 ('모빌리티 인재', 26),
 ('모빌리티 클래스', 27),
 ('모집', 28),
 ('모터스튜디오', 29),
 ('모터스튜디오 구축', 30),
 ('미래', 31),
 ('미래 모빌리티', 32),
 ('부산엑스포', 33),
 ('부산엑스포 유치', 34),
 ('삼성', 35),
 ('삼성 sk', 36),
 ('상승', 37),
 ('생태계', 38),
 ('생태계 확장', 39),
 ('스타트업', 40),
 ('스타트업 250개', 41),
 ('스타트업 생태계', 42),
 ('스타트업에', 43),
 ('양성', 44),
 ('연속', 45),
 ('연속 완주', 46),
 ('완주', 47),
 ('외국인', 48),
 ('외인', 49),
 ('유치', 50),
 ('육성', 51),
 ('인재', 52),
 ('인재 양성', 53),
 ('일자리', 54),
 ('재계', 55),
 ('재단', 56),
 ('전기차', 57),
 ('정몽구', 58),
 ('정몽구 재단', 59),
 ('정몽구재단', 60),
 ('제페토', 61),
 ('제페토 현대', 62),
 ('지원', 63),
 ('최태원', 64),
 ('출범', 65),


In [18]:
desCount = {} 
 
for word in description_slice:
    desCount[word] = desCount.get(word, 0) + 1
    desKeys = sorted(desCount.keys())

In [19]:
titleCount = {} 
 
for title in title_slice:
    titleCount[title] = titleCount.get(title, 0) + 1
    titleKeys = sorted(titleCount.keys())

In [20]:
for word in desKeys:
    if desCount[word] > 50:
        print(word + ':' + str(desCount[word]))

:951
2027년까지:85
24시:80
30일:121
CJ:51
LG,:77
N:102
SK,:101
공영운:60
교육:62
국내:210
글로벌:58
뉘르부르크링:111
대상으로:73
대학(원)생을:55
등:278
등이:67
맞은:51
모빌리티:100
미래:93
및:94
분야의:53
브랜드:56
사장,:73
삼성전자,:66
스타트업:138
시가총액:57
오는:71
위해:85
이공계:71
일자리:67
있다.:92
재단이:79
전기차:52
정몽구:139
주요:107
지난:68
차량:70
통해:85
투자:80
플랫폼:59
함께:79
현대:81
현대차:441
현대차,:139
현대차그룹과:116
현대차는:268
현대차의:139


In [21]:
for word in titleKeys:
    if titleCount[word] > 20:
        print(word + ':' + str(titleCount[word]))

'H-모빌리티:45
'뉘르부르크링:24
1000억:38
1000억원:21
24시:55
250개:29
5년간:47
7년:46
‘H-모빌리티:22
구축:30
국내:38
내구레이스:22
내구레이스':23
내구레이스’:21
뉘르부르크링:23
등:32
메타버스:43
모빌리티:62
모집:35
모터스튜디오:45
미래:53
부산엑스포:55
상승:40
생태계:24
스타트업:93
스타트업에:21
양성:50
연속:63
완주:66
외국인:24
유치:40
육성:23
인재:79
재단,:42
전기차:24
최태원:22
코스피:34
코스피,:61
클래스':44
클래스’:22
투자:66
플랫폼:25
현대:33
현대자동차,:24
현대차:84
현대차,:176
현대차그룹,:29
확장:21
